# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
file_path_list = glob.glob(os.path.join(filepath,'*.csv'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',\
                'level', 'location', 'sessionId', 'song', 'userId', 'username'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow(
            (row[0], row[2], row[3], int(row[4]), row[5], float(row[6]), 
             row[7], row[8], int(row[12]), row[13], int(row[16]), row[2]+" "+row[5]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance (local machine, in this case)
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [8]:
# We need query output as a Pandas Dataframe, defining a factory function for the same
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory

In [9]:
# row_factory has been set, now create a function to return Pandas dataframe after executing the query
def execute_query_pd(query, session):
    result = session.execute(query)
    return result._current_rows

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
def drop_tables(table_name='all'):
    """Drops the table specified in `table_name` parameter.
    
    If no table_name is provided, all the tables mentioned in `tables` list are dropped.
    """
    
    tables = ['artist_play_history', 'user_play_history', 'song_play_history']
    
    if table_name != 'all':
        tables = [table_name]
    
    for table_name in tables:
        session.execute(f'DROP TABLE IF EXISTS {table_name}')
        

In [11]:
# Drop all tables to reset the DB
drop_tables()

# TODO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

query_1 = """
    CREATE TABLE IF NOT EXISTS artist_play_history (
        session_id int,
        item_in_session int,
        artist varchar,
        song varchar,
        length float,
        PRIMARY KEY (session_id, item_in_session)
    )
"""

try:
    session.execute(query_1)
    print("Table 1 created")
except Exception as e:
    print(e)
    
# TODO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

query_2 = """
    CREATE TABLE IF NOT EXISTS user_play_history (
        user_id INT,
        session_id INT,
        item_in_session INT,
        artist varchar,
        song varchar,
        username varchar,
        PRIMARY KEY ((user_id, session_id), item_in_session)  
        -- user_id & session_id to uniquely identify the partition and item_in_session to sort the rows in cluster
    )
"""

try:
    session.execute(query_2)
    print("Table 2 created")
except Exception as e:
    print(e)
    
# TODO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query_3 = """
    CREATE TABLE IF NOT EXISTS song_play_history (
        song varchar,
        user_id INT,
        username varchar,
        PRIMARY KEY (song, user_id)
    )
"""

try:
    session.execute(query_3)
    print("Table 3 created")
except Exception as e:
    print(e)

Table 1 created
Table 2 created
Table 3 created


In [12]:
# Configure the INSERT statement for each table and their column-datatype map to avoid redundunt code for different tables
INSERT_QUERY_MAP = {
    "artist_play_history": [
        """INSERT INTO artist_play_history (session_id, item_in_session, artist, song, length) 
            VALUES(%s, %s, %s, %s, %s)""", 
        {'sessionId': int, 'itemInSession': int, 'artist': str, 'song': str, 'length': float}],
    "user_play_history": [
        """INSERT INTO user_play_history (user_id, session_id, item_in_session, artist, song, username) 
            VALUES(%s, %s, %s, %s, %s, %s)""", 
        {'userId': int, 'sessionId': int, 'itemInSession': int, 'artist': str, 'song': str, 'username': str}],
    "song_play_history": [
        """INSERT INTO song_play_history (song, user_id, username) 
            VALUES(%s, %s, %s)""", 
        {'song': str, 'userId': int, 'username': str}]
}

In [13]:
# Using the file that contains the data gathered from multiple event data files
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.DictReader(f)
    for line in csvreader:
        for table_name, query_and_values in INSERT_QUERY_MAP.items():
            # Get the INSERT query from `INSERT_QUERY_MAP`
            query = query_and_values[0]
            # Convert each column value to required datatype using type functions configured in `INSERT_QUERY_MAP`
            session.execute(query, [dtype(line[column]) for column, dtype in query_and_values[1].items()])

#### Do a SELECT to verify that the data have been inserted into each table

In [14]:
# QUERY: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

query_select_1 = """
    SELECT artist, song, length
    FROM artist_play_history
    WHERE session_id = 338 and item_in_session = 4
"""

df_query_1 = execute_query_pd(query_select_1, session)

df_query_1

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [15]:
# QUERY: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
query_select_2 = """
    SELECT artist, song, username AS user
    FROM user_play_history
    WHERE user_id = 10 AND session_id = 182
"""

df_query_2 = execute_query_pd(query_select_2, session)

df_query_2

,artist,song,user
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


In [40]:
# QUERY: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query_select_3 = """
    SELECT username AS user
    FROM song_play_history
    WHERE song = 'All Hands Against His Own'
"""

df_query_3 = execute_query_pd(query_select_3, session)

df_query_3

               user
0  Jacqueline Lynch
1      Tegan Levine
2      Sara Johnson


,user
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson


### Drop the tables before closing out the sessions

In [15]:
## TODO: Drop the table before closing out the sessions

In [16]:
drop_tables()

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()